In [1]:
import pymysql
import pandas as pd

# 공통
%matplotlib inline
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [2]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "select de_dt as 날짜 , sum(buy_am) as 금액 from 엘페이이용_view group by de_dt order by de_dt")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
# print(df.head(10))
df.head(10)

df.columns = ['date', 'price']
df = df.drop(['date'], axis = 1)
df.head(10)

,price
0,39209808
1,62239672
2,57365413
3,95785399
4,37415973
5,44468170
6,39459368
7,56869794
8,72284738
9,49765511


In [9]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
df_list = [start + relativedelta(days = x) for x in range(0, len(df)) ]

df['index'] = df_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
df.set_index( ['index'], inplace=True) 

# index 출력 X
df.index.name=None
df.tail(31)

,price
2021-12-01,77227183
2021-12-02,41526737
2021-12-03,69351467
2021-12-04,51247399
2021-12-05,68906456
2021-12-06,49715939
2021-12-07,35673851
2021-12-08,35659274
2021-12-09,48255147
2021-12-10,56768085


In [3]:
df = df[:-31]
df

,price
0,39209808
1,62239672
2,57365413
3,95785399
4,37415973
...,...
329,49831894
330,69518627
331,66978463
332,52584210


In [4]:
# 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기) p, q

import itertools

# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and d triplets
pdq = list( itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and d triplets
seasonal_pdq = [ (x[0], x[1], x[2], 365)  for x in pdq ]

print('Example of parameter combinations for Seasonal ARIMA ...')
print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[1]) )
print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[2]) )
print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[3]) )
print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[4]) )

Example of parameter combinations for Seasonal ARIMA ...
SARIMAX:  (0, 0, 1) x (0, 0, 1, 365)
SARIMAX:  (0, 0, 1) x (0, 1, 0, 365)
SARIMAX:  (0, 1, 0) x (0, 1, 1, 365)
SARIMAX:  (0, 1, 0) x (1, 0, 0, 365)


In [5]:
# price가 object임
df = df.astype('int')

In [6]:
df.dtypes

price    int32
dtype: object

In [16]:
# AIC(통계 모델을 성능을 측정하는 기준) - 낮을수록 좋음

select_candi = 10000000
param_candi = ( 0, 0, 0 )
param_seasonal_candi = ( 0, 0, 0)

count=0
end_count = len(pdq)

for param in pdq:   
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX( df['price'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False
                                           )
            results = mod.fit()
            count += 1
            if count <= 5:
                print('ARIMA{}x{}365 - AIC:{}'.format(param, param_seasonal, results.aic))
            
            if results.aic < select_candi:
                select_candi = results.aic
                param_candi = param
                param_seasonal_candi = param_seasonal
        except:
            continue
            
print(param_candi, param_seasonal_candi, select_candi) 

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Pytho

ARIMA(0, 0, 0)x(0, 0, 0, 365)365 - AIC:35170.91385503181
ARIMA(0, 0, 0)x(0, 0, 1, 365)365 - AIC:4.0


c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


ARIMA(0, 0, 0)x(0, 1, 0, 365)365 - AIC:2.0


c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be u

...

ARIMA(0, 0, 1)x(0, 0, 1, 365)365 - AIC:6.0


c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


MemoryError: Unable to allocate 344. MiB for an array with shape (367, 367, 335) and data type float64

: 

: 

In [7]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    df['price'],
    order = (0, 1, 0),
    seasonal_order = (0, 0, 1, 365),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


MemoryError: Unable to allocate 344. MiB for an array with shape (367, 367, 335) and data type float64

Exception ignored in: 'statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.reset_filter_method'
Traceback (most recent call last):
  File "statsmodels\tsa\statespace\_kalman_smoother.pyx", line 1014, in statsmodels.tsa.statespace._kalman_smoother.dKalmanSmoother.allocate_arrays
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 344. MiB for an array with shape (367, 367, 335) and data type float64


: 

: 

In [27]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 31 )

In [28]:
forecast_mean = pred_uc.predicted_mean
forecast_mean

2021-12-01    6.532627e+07
2021-12-02    6.216217e+07
2021-12-03    7.183506e+07
2021-12-04    6.048778e+07
2021-12-05    6.721317e+07
2021-12-06    5.956789e+07
2021-12-07    5.309096e+07
2021-12-08    5.785640e+07
2021-12-09    5.711356e+07
2021-12-10    7.268390e+07
2021-12-11    5.929910e+07
2021-12-12    5.825429e+07
2021-12-13    5.801978e+07
2021-12-14    6.264859e+07
2021-12-15    5.245900e+07
2021-12-16    5.939510e+07
2021-12-17    6.456092e+07
2021-12-18    6.225261e+07
2021-12-19    6.568931e+07
2021-12-20    4.714495e+07
2021-12-21    5.490944e+07
2021-12-22    5.140547e+07
2021-12-23    5.699818e+07
2021-12-24    5.676228e+07
2021-12-25    5.981387e+07
2021-12-26    5.887430e+07
2021-12-27    5.148668e+07
2021-12-28    5.115479e+07
2021-12-29    5.397929e+07
2021-12-30    6.766556e+07
2021-12-31    7.341607e+07
Freq: D, Name: predicted_mean, dtype: float64

In [29]:
forecast_mean.to_csv('lpoint_days_52.csv', header=['forecast'], index=False)

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error

actual = [77227183, 41526737, 69351467, 51247399, 68906456, 49715939, 35673851, 35659274, 48255147, 56768085, 69334618, 77053697, 57768387, 38486391, 31766615, 46015054, 51296762, 58203621, 54939758, 41869454, 51830618, 48838343 ,40257093, 44660225, 71313940, 67554989, 48002138, 38070210, 52093193, 47463341, 66341333]
forecast7 = [49187920, 48980260, 73889980, 70159370, 68338800, 57508330, 48684330, 49368140, 49149540, 74048980, 70308710, 68479080, 57640090, 48808090, 49484380, 49258720, 74151530, 70405040, 68569550, 57725070, 48887910, 49559360, 49329140, 74217670, 70467160, 68627910, 57779880, 48939390, 49607710, 49374560, 74260330]
forecast12 = [60194450, 59727190, 60096370, 63978410, 60916900, 60360360, 60625230, 57746520, 56553200, 58184600, 64327980, 66249350, 60497460, 61105150, 61715400, 65651500, 62602110, 62048290, 62313770, 59435190, 58241910, 59873310, 66016700, 67938060, 62186170, 62793860, 63404110, 67340210, 64290830, 63737010, 64002490]
forecast24 = [42571810, 57402750, 70629990, 65991940, 69439780, 63662350, 61327470, 60797030, 60636350, 71057290, 69763530, 72313200, 59118510, 63139330, 65436690, 65491610, 57723400, 68472040, 72265290, 55796120, 48657390, 52679190, 64140600, 60506090, 62875560, 53516480, 64772240, 64953890, 68565660, 63316630, 60893780]
forecast52 = [65326270, 62162170, 71835060, 60487780, 67213170, 59567890, 53090960, 57856400, 57113560, 72683900, 59299100, 58254290, 58019780, 62648590, 52459000, 59395100, 64560920, 62252610, 65689310, 47144950, 54909440, 51405470, 56998180, 56762280, 59813870, 58874300, 51486680, 51154790, 53979290, 67665560, 73416070]

rmse = np.sqrt(mean_squared_error(actual, forecast7))

print(rmse)

nan


C:\Users\ASIA-08\AppData\Local\Temp\ipykernel_14428\3814821178.py:10: RuntimeWarning: invalid value encountered in sqrt
  rmse = np.sqrt(mean_squared_error(actual, forecast7))


In [20]:
mse = mean_squared_error(actual, forecast12)

# mse 값이 음수라면 절대값으로 만들어준 뒤 rmse를 구하면 됨
mse = abs(mse)

rmse = np.sqrt(mse)
rmse

3476.7061348907673